In [1]:
import numpy as np
import pandas as pd
from datetime import date
import random
import math

In [2]:
def roll_dice():
    return(np.random.randint(1,7))

def shuffle(df):
    return(df.sample(frac=1).reset_index(drop=True))

def take_cards(df, df_total, n=1):
    if (df.shape[0] == 0):
        df = shuffle(df_total)
    card = df.head(n)
    df.drop(index=df.index[:n], axis=0, inplace=True)
    return(df, card)

In [16]:
class Battle:
    def __init__(self, player, monster):
        self.monster = monster
        self.player = player
    def before_battle(self):
        for i in self.monster.before_battle:
            eval(i)
    def fight(self):
        if (self.player.bonus > self.monster.level):
            self.get_reward()
        else:
            while (self.player.decision['one_hot'] > self.player.coef['one_hot'] 
                   and '<there are one hots in the hand>'):
                # select what cards and how many to use?
                # how to actualize????
                self.player.decision['one_hot'] -= (self.player.decision['one_hot'] *
                                                    self.player.learning_rate['one_hot'])
                # how to actualize????
                if (self.player.bonus > self.monster.level):
                    self.get_reward()
                    break
            if (self.player.bonus < self.monster.level):        
                self.escape_after()
    
    def escape_after(self):
        for i in self.monster.even_if_escaped:
            eval(i)
        if roll_dice()+self.player.escape < self.monster.escape:
            self.get_bad_stuff()
    def get_reward(self):
        self.player.update_level(n=monster.reward_level)
        self.player.update_hand(n=monster.reward_treasure,
                                new=take_cards(df, df_total, n=monster.reward_treasure))
        # !!!! somehow pass card decks + remove from deck -- maybe create another class:)
    def get_bad_stuff(self):
        for i in self.monster.bad_stuff:
            eval(i)

In [4]:
class Monster:
    def __init__(self, df):
        self.name = df.name.iloc[0, ]
        self.level = df.level.iloc[0, ]
        self.undead = df.undead.iloc[0, ]
        self.before_battle = str(df.before_battle.to_list()[0]).split(';')
        self.even_if_escaped = str(df.even_if_escaped.to_list()[0]).split(';')
        if df.escapable.iloc[0, ]:
            self.escape = 4
        else:
            self.escape = 10e7
        self.bad_stuff = df.bad_stuff.iloc[0, ].split(';')
        self.reward_level = df.reward_level.iloc[0, ]
        self.reward_treasure = df.reward_treasure.iloc[0, ]
        self.multiplier = 1
    def update_level(self, n, condition=None, update_type=None):
        if update_type == 'monster_buff' and self.multiplier > 1:
            n *= self.multiplier
        if condition is None:
            self.level += n
        elif eval(condition):
            self.level += n
    def kill(self, condition=None, treasures=True, levels=True):
        if condition is None:
            self.level = -10e7
        elif eval(condition):
            self.level = -10e7
            
        if not treasures:
            self.reward_treasure = 0
        if not levels:
            self.reward_level = 0
    def update_escape(n, condition=None):
        if condition is None:
            self.escape += n
        elif eval(condition):
            self.escape += n
    def update_multiplier(n):
        self.multiplier += n
    def update_treasures(n, condition=None):
        if condition is None:
            self.reward_treasure += n
        elif eval(condition):
            self.reward_treasure += n
        
        if self.reward_treasure < 0:
            self.reward_treasure = 0
    def update_reward_level(n, condition=None):
        if condition is None:
            self.reward_level += n
        elif eval(condition):
            self.reward_level += n

In [5]:
class Munchkin:
    empty_treasure = pd.DataFrame(
                dict(name=[], type=[], eq_type=[], big=[], restriction=[],
                     bonus=[], hands=[], buffs=[], price=[], when_lose=[])
            )
    empty_hand = pd.DataFrame(
                dict(name=[], type=[], eq_type=[], big=[], restriction=[],
                     bonus=[], hands=[], buffs=[], price=[], when_lose=[])
            )
    def __init__(self, name='John Doe', sex='female', level=1, pclass=[], race=['human'],
                 hand=empty_hand, equiped=empty_treasure):
        self.name = name
        self.sex = sex
        self.pclass = pclass
        self.race = race
        self.level = level
        self.escape = 0
        self.bonus = level
        self.equiped = equiped
        self.hand = hand
        self.limits = dict(armor=1, boots=1, head=1, hands=2, big=1, cards=5)
        self.statuses = pd.DataFrame()
    def update_level(self, n, condition=None, update_type='fight'):
        if condition is None:
            self.level += n
        elif eval(condition):
            self.level += n
        
        if self.level < 1:
            self.level = 1
    def update_pclass_race(self, what, n=1, which=None, make_decision=False):
        if n == -1 and make_decision:
            which = make_decision('lose_pclass_race', choices='self.'+what)
            eval('self.'+what+'.pop(self.'+what+'.index(which))')
        elif n == -1 and not(make_decision):
            eval('self.'+what+'.pop(which)')
        elif n == -2:
            eval('self.'+what+'.clear()')
        elif n > 0:
            which = make_decision('add_pclass_race', choices='self.'+what)
            eval('self.'+what+'.extend(which)')
        if what=='race' and len(self.race) == 0:
            self.race = ['human']
    def update_sex(self, change=False):
        if change == False:
            if self.sex == 'male':
                self.sex = 'female'
            else:
                self.sex = 'male'
        else:
            self.sex = 'none'
    def update_limit(self, which, n):
        self.limits[which] += n
    def update_hand(self, n=None, condition=None, everything=False, make_decision=True,
                    give=False, make_decision_give=False, new=None):
        if n > 0:
            self.hand = pd.concat([self.hand, new]).reset_index(drop=True)
        else:
            if everything:
                hand_shortened = empty_hand
                hand_to_give = self.hand
                make_decision = False
            elif condition is not None:
                hand_shortened = self.hand.loc[~eval(condition), :].reset_index(drop=True).copy()
                hand_to_give = self.hand.loc[eval(condition), :].reset_index(drop=True).copy()
            else:
                hand_shortened = self.hand
                hand_to_give = empty_hand
        
            if make_decision:
                which = self.make_decision('lose_card', choices='hand', n=n)
                hand_shortened = hand_shortened.iloc[~which, :].reset_index(drop=True)
                hand_to_give = pd.concat(
                    [hand_to_give,hand_shortened.iloc[which, :].reset_index(drop=True)]
                )
            if give:
                self.give(hand_to_give, make_decision_give)
            self.hand = hand_shortened
    def lose_equiped(self, n=None, condition=None, everything=False, make_decision=True,
                     give=False, make_decision_give=False):
        if everything:
            equipted_shortened = empty_treasure
            equipted_to_give = self.equiped
            make_decision = False
        elif condition is not None:
            equipted_shortened = self.equiped.loc[~eval(condition), :].reset_index(drop=True).copy()
            equipted_to_give = self.equiped.loc[eval(condition), :].reset_index(drop=True).copy()
        else:
            equipted_shortened = self.equiped
            equipted_to_give = empty_treasure
        
        if make_decision:
            which = self.make_decision('lose_card', choices='equiped', n=n)
            equipted_shortened = equipted_shortened.iloc[~which, :].reset_index(drop=True)
            equipted_to_give = pd.concat(
                [equipted_to_give,
                 equipted_shortened.iloc[which, :].reset_index(drop=True)]
            )
        
        if give:
            self.give(equipted_to_give, make_decision_give)
        
        tmp_lst = equipted_to_give.when_lose[equipted_to_give.when_lose.notnull()]
        tmp_lst = list(map(lambda x: x.split(';'), tmp_lst))
        tmp_lst = [item for sublist in tmp_lst for item in sublist]
        for i in tmp_lst:
            eval(i)
        self.equiped = equipted_shortened
    def equip(self):
        what = self.make_decision('equip_what')
        tmp_to_equip = self.hand.iloc[what, :].reset_index(drop=True).copy()
        tmp_to_equip_con = pd.concat([self.equipted, tmp_to_equip])
        check_limits = (
            tmp_to_equip_con[tmp_to_equip_con.eq_type == 'armor'].shape[0] < self.limits['armor'] and
            tmp_to_equip_con[tmp_to_equip_con.eq_type== 'boots'].shape[0] < self.limits['boots'] and
            tmp_to_equip_con[tmp_to_equip_con.eq_type == 'head'].shape[0] < self.limits['head'] and
            tmp_to_equip_con[tmp_to_equip_con.eq_type == True].shape[0] < self.limits['big'] and
            tmp_to_equip_con.eq_type.hands.sum() < self.limits['hands']
        )
        tmp_lst = tmp_to_equip.loc[tmp_to_equip.eq_type != 'one_shot'].restriction
        tmp_lst = tmp_lst[tmp_lst.notnull()].to_list()
        tmp_lst = list(map(lambda x: x.split(';'), tmp_lst))
        tmp_lst = [item for sublist in tmp_lst for item in sublist]
        restrictiontions_check = []
        for i in tmp_lst:
            restrictiontions_check.append(eval(i))
        
        if check_limits and all(restrictiontions_check):
            #eval effects:
            tmp_lst = tmp_to_equip.loc[tmp_to_equip.eq_type != 'one_shot'].buffs
            tmp_lst = tmp_lst[tmp_lst.notnull()].to_list()
            tmp_lst = list(map(lambda x: x.split(';'), tmp_lst))
            tmp_lst = [item for sublist in tmp_lst for item in sublist]
            for i in tmp_lst:
                eval(i)
            self.equipted = tmp_to_equip_con
            self.hand = self.hand.iloc[~self.hand.index.isin(what), :].reset_index(drop=True)
    def add_status(self, name, when_vanish):
        # add statuses
        pass
    def update_bonus(self, n):
        self.bonus += n
    def update_escape(self, n):
        self.escape += n
    def die(self, marauder=True):
        equipted_to_give = pd.concat([self.hand, self.equiped])
        self.equiped = empty_treasure
        self.hand = empty_hand
        if marauder:
            self.give(equipted_to_give)
    def give(self, what, make_decision=False):
        pass
    def upgrade_equipted(self):
        pass
    def make_decision(self, what, choices=None, n=None):
        if what == 'fight':
            decision = np.where(roll_dice()>=3, True, False)
        elif what == 'bad_stuff':
            decision = np.where(roll_dice()>=3, True, False)
        elif what == 'lose_card':
            if choices == 'hand':
                decision = [random.randint(0, self.hand.shape[0]-1) for i in range(n)]
            elif choices == 'equiped':
                decision = [random.randint(0, self.equiped.shape[0]-1) for i in range(n)]
        elif what == 'lose_pclass_race':
            if choices == 'race':
                decision = random.randint(0, len(self.race))
            elif choices == 'pclass':
                decision = random.randint(0, len(self.pclass))
        elif what == 'give':
            if choices == 'hand':
                decision = [random.randint(0, self.hand.shape[0]-1) for i in range(n)]
            elif choices == 'equiped':
                decision = [random.randint(0, self.equiped.shape[0]-1) for i in range(n)]
        else:
            decision = []
        return decision
        # add_pclass_race -- which pclass / race to add
        # give -- which card(s) to give
        # one_shot -- use or not
        # one_shot-what -- what to use
        # sell - to sell or not
        # sell_what -- what to sell
        # equip - to equip or not
        # equip_what

In [6]:
GLOBAL_STUFF_FLAME = ['hot_pepper_sauce', 'flaming_armor', 'oil_of_boiling']
GLOBAL_STUFF_BELOW_ARMOR = ['spiky_knees', 'kneepads_of_allure', 'spiked_codpiece']
GLOBAL_LEVEL_MIN = 0
GLOBAL_LEVEL_MAX = 0

monsters = pd.read_excel('example.xlsx', sheet_name='monsters')
curses = pd.read_excel('example.xlsx', sheet_name='curses')
races = pd.read_excel('example.xlsx', sheet_name='races')
classes = pd.read_excel('example.xlsx', sheet_name='classes')
doors_other = pd.read_excel('example.xlsx', sheet_name='doors_other')
dect_doors = pd.concat([monsters, curses, races, classes, doors_other]).reset_index(drop=True)
deck_treasures = pd.read_excel('example.xlsx', sheet_name='treasures')

dect_doors = shuffle(dect_doors)
deck_treasures = shuffle(deck_treasures)

dect_doors_current = dect_doors.copy()
deck_treasures_current = deck_treasures.copy()

player_1 = Munchkin()

In [7]:
dect_doors_current, door_card = take_cards(dect_doors_current, dect_doors, 1)
door_card['type'].tolist()[0] == 'monster'

True

In [57]:
current_monster = Monster(door_card)
current_player = player_1
current_monster.level
#current_player.pclass = ['bard']

11.0

In [55]:
eval("")

In [56]:
current_monster.level

8.0

In [58]:
current_monster.before_battle = [
    "self.monster.update_level(-3) if 'bard' in self.player.pclass else ''"
]
current_battle = Battle(current_player, current_monster)
#["self.monster.update_level(-3, condition='\\'bard\\'  in self.player.pclass')"]

In [67]:
current_battle.before_battle()
current_monster.level

-16.0

In [ ]:
#self.decisions = dict(fight=1, one_hot=1, auto_escape=1)
#self.coefs = dict(fight=0.5, one_hot=0.5, auto_escape=0.1)
#self.learning_rate = dict(fight=0.005, one_hot=0.005, auto_escape=0.005)
# Set queue:
current_player = player_1
fight_coef = 0.5

# Prepare:
# sell
# equip

# Main game:
dect_doors_current, door_card = take_cards(dect_doors_current, dect_doors, 1)
if door_card['type'].tolist()[0] == 'monster':
    print('Battle begins..')
    # current_monster = Monster(door_card)
    # current_player = Battle(current_player, current_monster).escape_before().fight().player
elif door_card['type'].tolist()[0] == 'curse':
    print('Curse..')
    #current_player.curse(door_card)
# if bozestwennoye wmeszatelstwo
else:
    print('Taking door card..')
    current_player.update_hand(n=1, h_type='doors', new=door_card)
    dect_doors.drop(dect_doors[dect_doors.name!=door_card.name[0]].index, inplace=True)
    if (current_player.decisions['fight'] > fight_coef and 
    'monster' in current_player.hand['doors']['type'].tolist()):
        print('Battle begins..')
        current_player = battle(current_player, 'which to fight???') # !!!!!!!!!!!!!!!!!!!!
    else:
        print('Taking door card..')
        dect_doors_current, door_card = take_cards(dect_doors_current, dect_doors, 1)
        current_player.update_hand(n=1, h_type='doors', new=door_card)
        dect_doors.drop(dect_doors[dect_doors.name!=door_card.name[0]].index, inplace=True)
        
# Finalize:
# Update decisions:
# Check convergence: